In [1]:
import pandas as pd
data = pd.read_csv('qa.csv')

In [2]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
test_data = [
    "What is the population of Egypt?",
    "What is the poulation of egypt",
    "How long is a leopard's tail?",
    "Do you know the length of leopard's tail?",
    "When polar bears can be invisible?",
    "Can I see arctic animals?",
    "some city in Finland"
]

In [3]:
data

,Question,Answer
0,Who determined the dependence of the boiling o...,Anders Celsius
1,Are beetles insects?,Yes
2,Are Canada 's two official languages English a...,yes
3,What is the population of Egypt?,more than 78 million
4,What is the biggest city in Finland?,Greater Helsinki
5,What is the national currency of Liechtenstein?,Swiss franc
6,Can polar bears be seen under infrared photogr...,Polar bears are nearly invisible under infrare...
7,When did Tesla demonstrate wireless communicat...,1893
8,What are violins made of?,different types of wood
9,How long is a leopard's tail?,60 to 110cm


In [4]:
import re
def getNaiveAnswer(q):
    # regex helps to pass some punctuation signs
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Sorry, I didn't get you.", 0, ""
getResults(test_data, getNaiveAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1
1,What is the poulation of egypt,,"Sorry, I didn't get you.",0
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1
3,Do you know the length of leopard's tail?,,"Sorry, I didn't get you.",0
4,When polar bears can be invisible?,,"Sorry, I didn't get you.",0
5,Can I see arctic animals?,,"Sorry, I didn't get you.",0
6,some city in Finland,,"Sorry, I didn't get you.",0


In [5]:
from Levenshtein import ratio
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.8:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1.000000
1,What is the poulation of egypt,What is the population of Egypt?,more than 78 million,0.935484
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1.000000
3,Do you know the length of leopard's tail?,How long is a leopard's tail?,"Sorry, I didn't get you.",0.657143
4,When polar bears can be invisible?,Can polar bears be seen under infrared photogr...,"Sorry, I didn't get you.",0.517647
5,Can I see arctic animals?,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.426230
6,some city in Finland,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.642857


In [6]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer2)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1.000000
1,What is the poulation of egypt,What is the population of Egypt?,more than 78 million,0.935484
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1.000000
3,Do you know the length of leopard's tail?,How long is a leopard's tail?,60 to 110cm,0.657143
4,When polar bears can be invisible?,Can polar bears be seen under infrared photogr...,Polar bears are nearly invisible under infrare...,0.517647
5,Can I see arctic animals?,What is the biggest city in Finland?,Greater Helsinki,0.426230
6,some city in Finland,What is the biggest city in Finland?,Greater Helsinki,0.642857
